# Balanced Binary Search Trees

- **Search:** $O(\log n)$
- **Select:** $O(\log n)$
- **Min/Max:** $O(\log n)$
- **Pred/Succ:** $O(\log n)$
- **Rank:** $O(\log n)$
- **Insert/Delete:** $O(\log n)$
- **Output in sorted order:** $O(n)$

# Binary Search Trees

* One note per key.
* Basic version, node has left child pointer, right child pointer, and parent pointer.
* **Search Tree Property:** Key in each node must be greater than or equal any key in the left subtree and smaller than or equal any key in the right subtree.
* Since BST doesn't have to be balanced, height can be between $\log n$ and $n$.

## Operations

* **Search(k):**
    * Traverse to left or right subtree depending on k beeing smaller or larger than the search key.
* **Insert(k):**
    * Search for k and insert at it's position by rewiring pointers.
* **Delete(k):**
    * Search for k.
    * If no children: Delete k.
    * One child: Splice out k's node.
    * Two children: Compute k's predecessor l, swap k and l, delete dangling pointers.
* **Predecessor(k):**
    * Search for k.
    * If k's left subtree is not empty, return max key in left subtree.
    * Follow parent pointer until key is less than k.

In [1]:
class Node:
    
    def __init__(self, key, parent=None, left=None, right=None):
        self.key = key
        self.parent = parent
        self.left = left
        self.right = right
        
    def add(self, key):
        if self.key > key:
            if not self.left:
                self.left = Node(key, self)
            else:
                self.left.add(key)
        else:
            if not self.right:
                self.right = Node(key, self)
            else:
                self.right.add(key)
    
    def get(self, key):
        if self.key == key:
            return self
        if self.key > key:
            if not self.left:
                return None
            return self.left.get(key)
        else:
            if not self.right:
                return None
            return self.right.get(value)
        
    def delete(self, key):
        if self.key > key:
            if self.left:
                self.left.delete(key)
        elif self.key < key:
            if self.right:
                self.right.delete(key)
        else:
            if self.left and self.right:
                successor = self.right.find_min()
                self.key = successor.key
                successor.delete(successor.key)
            elif self.left:
                self.splice(self.left)
            elif self.right:
                self.splice(self.right)
            else:
                self.splice()
                
    def splice(self, new_node=None):
        if self.parent:
            if self == self.parent.left:
                self.parent.left = new_node
            else:
                self.parent.right = new_node
        if new_node:
            new_node.parent = self.parent
    
    def find_min(self):
        if self.left:
            return self.left.find_min()
        return self
    
    
def inorder(node):
    if node:
        inorder(node.left)
        print(node.key)
        inorder(node.right)
        
def preorder(node):
    if node:
        print(node.key)
        preorder(node.left)
        preorder(node.right)

def postorder(node):
    if node:
        postorder(node.right)
        postorder(node.left)
        print(node.key)

def print_tree(root):
    seperator = 40
    current_level = [root]
    while current_level:
        seperator = int(seperator / 1.7)
        spaces = ' ' * seperator
        level = [spaces + str(node.key) if node else spaces + 'N' for node in current_level]
        print(''.join(level))
        next_level = []
        for n in current_level:
            if n:
                next_level.append(n.left if n.left else None)
                next_level.append(n.right if n.right else None)
            current_level = next_level

In [2]:
root = Node(5)
[root.add(node) for node in [2, 8, 1, 10, 4, 6]]
print_tree(root)
print('\n\nDelete 10 (No children):')
root.delete(10)
print_tree(root)
print('\n\nDelete 8 (One child):')
root.delete(8)
print_tree(root)
print('\n\nDelete 2 (Two children):')
root.delete(2)
print_tree(root)

                       5
             2             8
       1       4       6       10
    N    N    N    N    N    N    N    N


Delete 10 (No children):
                       5
             2             8
       1       4       6       N
    N    N    N    N    N    N


Delete 8 (One child):
                       5
             2             6
       1       4       N       N
    N    N    N    N


Delete 2 (Two children):
                       5
             4             6
       1       N       N       N
    N    N


## Red-Black Trees

### Invariants:

1. Each node is either red or black.
2. Root node is black.
3. No two red nodes in a row. If the node is red, it's children are black.
4. Every root -> null path must have the same number of black nodes.

$\Rightarrow$ RB Trees are balanced and have height $\leq 2 \log_2(n + 1)$

### Operations:

* Tree is balanced by doing rotations on insert/delete (can be done in $O(1)$).
    * *Intuituin:* Node is inserted as in BST, then recolor and/or perform rotations until invariants are restored.
* Rotation on node x / y. A, B, C are arbitrary subtrees (A < x, y; y > B > x; C > x, y ).
```
    P              P
    |              |
    x      <=>     y
   / \            / \
  A   y          x   C
     / \        / \
    B   C      A   B
```

**Insert(x):**
1. Insert x in BST.
2. Try coloring x red.
3. If x's parent (y) is black, done.
4. If y is red, y has black parent w.
    * **Case 1:** The other child z of w (y's neighbor) is also red.
        * 1) Recolor y, z black and w red (does not break invariant 1).
        * 2) This either restores invariant 3 or propagates the double red upwards.
        * 3) If propagated upwards, iteratively resolve. This can only happen $O(\log n)$ times.
        * 4) If root is reached, recolor it black (does not break invariant 4).
    * **Case 2:** Let x,y be the current double red, x the deeper node. Let w be x's grandparent (y's parent). Suppose w's other child (!= y) is Null or a black node z.
        * => We can eliminate the double red in $O(1)$ via 2-3 rotations and reorderings.